In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

C:\ProgramData\Anaconda3\lib\site-packages\requests\__init__.py:80: RequestsDependencyWarning: urllib3 (1.24.3) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
# Dataset
mnist = input_data.read_data_sets('../data/mnist/', one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../data/mnist/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../data/mnist/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ../data/mnist/t10k-images-idx3-ubyte.gz
Extracting ../data/mnist/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
# Settings
learning_rate = 0.1
training_epochs = 10
batch_size = 64
n_hidden_1 = 128
n_hidden_2 = 256
n_input = 784
n_classes = 10

In [4]:
# Graph Definition
g = tf.Graph()
with g.as_default() as g:
    tf_x = tf.placeholder(tf.float32, [None, n_input], name='features')
    tf_y = tf.placeholder(tf.float32, [None, n_classes], name='targets')

    weights = {
        'h1': tf.Variable(tf.truncated_normal([n_input, n_hidden_1], stddev=0.1)),
        'h2': tf.Variable(tf.truncated_normal([n_hidden_1, n_hidden_2], stddev=0.1)),
        'out': tf.Variable(tf.truncated_normal([n_hidden_2, n_classes], stddev=0.1))
    }
    biases = {
        'b1': tf.Variable(tf.zeros([n_hidden_1])),
        'b2': tf.Variable(tf.zeros([n_hidden_2])),
        'out': tf.Variable(tf.zeros([n_classes])),
    }

    # Multillayer perceptron
    layer_1 = tf.add(tf.matmul(tf_x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']

    # Loss and optimizer
    loss = tf.nn.softmax_cross_entropy_with_logits_v2(logits=out_layer, labels=tf_y)
    cost = tf.reduce_mean(loss, name='cost')
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    train = optimizer.minimize(cost, name='train')

    # Prediction
    correct_prediction = tf.equal(tf.argmax(tf_y, 1), tf.argmax(out_layer, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name='accuracy')

Instructions for updating:
Colocations handled automatically by placer.


In [5]:
# Training & Evaluation
with tf.Session(graph=g) as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(training_epochs):
        avg_cost = 0.0
        total_batch = mnist.train.num_examples // batch_size
        for i in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            _, c = sess.run([train, cost], feed_dict={tf_x: batch_x, tf_y: batch_y})
            avg_cost += c
        train_acc = sess.run(accuracy, feed_dict={tf_x: mnist.train.images, tf_y: mnist.train.labels})
        valid_acc = sess.run(accuracy, feed_dict={tf_x: mnist.validation.images, tf_y: mnist.validation.labels})
        print("Epoch: %03d | AvgCost: %.3f" % (epoch + 1, avg_cost / (i + 1)), end="")
        print(" | Train/Valid ACC: %.3f/%.3f" % (train_acc, valid_acc))

    test_acc = sess.run(accuracy, feed_dict={tf_x: mnist.test.images, tf_y: mnist.test.labels})
    print('Test ACC: %.3f' % test_acc)

Epoch: 001 | AvgCost: 0.345 | Train/Valid ACC: 0.946/0.953
Epoch: 002 | AvgCost: 0.160 | Train/Valid ACC: 0.963/0.963
Epoch: 003 | AvgCost: 0.116 | Train/Valid ACC: 0.973/0.970
Epoch: 004 | AvgCost: 0.091 | Train/Valid ACC: 0.977/0.972
Epoch: 005 | AvgCost: 0.073 | Train/Valid ACC: 0.981/0.973
Epoch: 006 | AvgCost: 0.062 | Train/Valid ACC: 0.986/0.976
Epoch: 007 | AvgCost: 0.051 | Train/Valid ACC: 0.987/0.978
Epoch: 008 | AvgCost: 0.044 | Train/Valid ACC: 0.990/0.977
Epoch: 009 | AvgCost: 0.036 | Train/Valid ACC: 0.992/0.979
Epoch: 010 | AvgCost: 0.030 | Train/Valid ACC: 0.994/0.980
Test ACC: 0.976
